Generalized World Coordinate System (GWCS)
==========================================

This section highlights one of the most powerful capabilities of ASDF; namely
its ability to save arbitrarily complex coordinate transformations with great 
flexibility. This includes the ability to:

- combine transformations using arithmetic operators
- combine transformations in series
- for an arbitrary number of dimensions
- define intermediate coordinates (e.g., the slit plane of a spectrograph)
- parameterize transformations using the parameters as extra dimensions
  (e.g., spectral order, position across a slit, date, etc.)

Contrast this with the FITS WCS system, which works well in imaging and spectra
for standard projections and dispersions, but poorly when dealing with raw data
cointaining complex distortions, or discontinuous transforms (e.g., IFUs), and 
particularly for slitless spectroscopy.

For HST, to achieve sub 0.01 pixel accuracy, 3 different distortion components 
had to be modeled, which were impossible to represent within the FITS WCS 
framework.

We are not able to convey the full capabilities in a few minutes of a tutorial.
This tutorial will illustrate some basics with an imaging example.

We will start with a simple projection and then augment with a distortion model.

The simple projection replicates the basic FITS capabilities using a tangent
projection followed by the appropriate transformation to celestial coordinates.

This involves identifying the point in the detector array that will be the tangent
point, applying the appropriate offset and scaling before applying the tangent
projection, and then transforming the resulting angular coordinates to celestial
coordinates. Schematically:

- Offset detector coordinates to make tangent point in detector have 0, 0 coordinates
- Scale resulting array coordinates to corresponding angular scale.
- Rotate detector coordinates so that north is up
- Apply inverse tangent projection.
- Transform resulting spherical coordinates to corresponding reference point
  in the celestial coordinate system with the appropriate position angle.

These operations are performed using astropy modeling package models.

In [5]:
from astropy.modeling import models
from gwcs import wcs
from gwcs import coordinate_frames as cf
from astropy import coordinates as coord
from astropy import units as u

In [6]:
# For simplicity we will assume that the detector y-axis is aligned with north, so no 
# rotation of detector coordinates is necessary.
# First step is to defined individual transformation models.
# We assume the detector array is 2000 x 2000 and the tangent point is at (1000, 1000)

# The following constructs a 2D model that shifts both input x and y coordinates by 1000
shift = models.Shift(-1000) & models.Shift(-1000)
# The following constructs a 2D model that scales both input x and y coordinates
# such that the center pixel is 0.1 arcsec in size
scale = models.Scale(0.1 / 3600.) & models.Scale(0.1 / 3600.)
# The following applies an inverse tangent projection
tanproj = models.Pix2Sky_TAN()
# The following moves the spherical coordinates so that the (0, 0) coordinates are moved
# to the supplied RA & Dec coordinates (in degrees), in this case RA = 30, Dec = 45
celest_rot = models.RotateNative2Celestial(30., 45., 180.) # last arg is always 180.

# The following is the net transformation from pixel coordinates to celestial coordinates
transform = shift | scale | tanproj | celest_rot

In [7]:
# Now we define the frames of reference for the WCS
detector_frame = cf.Frame2D(name='detector', axes_names=('x','y'), unit=(u.pix, u.pix))
sky_frame = cf.CelestialFrame(reference_frame=coord.ICRS(), unit=(u.deg, u.deg))
wcsobj = wcs.WCS([(detector_frame, transform), (sky_frame, None)])

In [8]:
wcsobj(1000, 1000)

(29.999999999999993, 45.00000000000001)

In [9]:
wcsobj(1000, 1001)

(29.999999999999993, 45.000027777777774)

In [10]:
1/3600,

(0.0002777777777777778,)

In [13]:
wcsobj(1000, 1000 + 36000 * 5)

(29.999999999999996, 49.98736528875502)

Spectral Example
----------------

We will consider building a WCS for a slitless spectrograph. For simplicity's sake,
This will be done as a 1D spectrograph, although the same concepts can be generalized
to 2D. This is an interesting case since given a pixel in the detector mapping it to
a wavelength is not possible unless the position of the source is known. In other 
words, one must know either position or wavelength, to determine the other. Generally
for the slitless the position is determined by other means in order for the wavelenth
to be determined.

This WCS will be based on a simple spectrograph design, taken from this 
[site](http://www.astrosurf.com/buil/us/stage/calcul/design_us.htm), since
the relevant design parameters make it simple to generate the WCS.

The relevant parameters are:

telescope focal length: 1000 mm
collimator focal length: 96 mm
camera focal length: 50 mm
grating density: 600 grooves/mm
nominal grating incident angle: 28 degrees

The optical axis of the collimator is presumed to correspond to 0 relative
angular coordinate from the telescope. Given a relative spatial offset in arcseconds
the computed incident grating angle becomes:

incident_grating_angle = (relative_spatial_angle * telescope_focal_length / collimator_focal_length) + design_incident_grating angle